In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from Bio import Entrez
import xml.etree.ElementTree as ET
import os
import threading

In [2]:
import pickle
with open("store_urls.pkl", 'rb') as f:
    urls = pickle.load(f)
print(len(urls))

1820


In [5]:
for i in range(len(urls)):
    if urls[i] == None:
        urls.pop(i)
print(len(urls))

1761


In [ ]:
class ScrapeThread(threading.Thread):
    def __init__(self, url, j):
        threading.Thread.__init__(self)
        self.url = url
        self.j = j

    def run(self):
        driver = webdriver.Chrome()
        try:
            driver.get(self.url)
            time.sleep(2)
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')

            # Find the article tag
            article = soup.find('article')

            # Find all p and header (h1 to h6) tags inside the article tag
            if article:
                elements = article.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

                # Collect the text content of each element in the correct order
                full_text = [element.text.strip() for element in elements]
                full_text = "\n".join(full_text)

                # Join the collected text into a single string with newlines
                if full_text.strip():
                    print("Text extracted.")
                    with open(f'C:\\Users\\Ajay Kanna\\Desktop\\UGRC\\extract abstract\\Lancet\\full_text_stored\\extracted_text_{self.j}.txt', 'w', encoding='utf-8') as file:
                        file.write(full_text)
            else:
                print("No article tag found")
        finally:
            driver.quit()

threads = []
batch = 5
for i in range(0, len(urls), 4):
    for url in urls[i: i+4]:
        t = ScrapeThread(url, )
        t.start()
        threads.append(t)

    for t in threads:
        t.join()


In [6]:
import threading
from queue import Queue
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

class ScrapeThread(threading.Thread):
    def __init__(self, queue, semaphore):
        threading.Thread.__init__(self)
        self.queue = queue
        self.semaphore = semaphore
        print("Thread initialized")

    def run(self):
        while True:
            print("Thread running")
            decision = False
            url, j = self.queue.get()
            if url is None:
                break

            '''chrome_options = Options()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_service = Service('chromedriver.exe')  # Path to chromedriver
            driver = webdriver.Chrome(service=chrome_service, options=chrome_options)'''

            driver = webdriver.Chrome()
            try:
                driver.get(url)
                time.sleep(4)
                
                html = driver.page_source
                soup = BeautifulSoup(html, 'lxml')

                # Find the article tag
                article = soup.find('article')

                # Find all p and header (h1 to h6) tags inside the article tag
                if article:
                    elements = article.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

                    # Collect the text content of each element in the correct order
                    full_text = [element.text.strip() for element in elements]
                    full_text = "\n".join(full_text)

                    full_text = filter_text(full_text)

                    # Join the collected text into a single string with newlines
                    if full_text.strip():
                        print(f"Text extracted for URL {j}.")
                        with open(f'C:\\Users\\Ajay Kanna\\Desktop\\UGRC\\extract abstract\\Lancet\\full_text_stored\\extracted_text_{j}.txt', 'w', encoding='utf-8') as file:
                            file.write(full_text)
                else:
                    print(f"No article tag found for URL {j}.")
            except Exception as e:
                print(f"Error processing URL {j}: {e}")
                
            finally:
                print("Releasing semaphore and marking task done")
                driver.quit()
                self.queue.task_done()
                self.semaphore.release()


def filter_text(text):
    # Remove references like (6) or [6] or References (6)
    text = re.sub(r'\bReferences? \(\d+\)', '', text)
    text = re.sub(r'\bReferences\b', '', text)
    # Remove "Cited by" with any number
    text = re.sub(r'\bCited by \(\d+\)', '', text)
    text = re.sub(r'\bCited by \d+', '', text)
    text = re.sub(r'\bCited by\b', '', text)

    if len(text.split()) < 50:
        text = ""
    
    return text

def main(urls):
    queue = Queue()
    semaphore = threading.Semaphore(10)  # Limit the number of concurrent threads to 4

    threads = []
    for i in range(10):
        t = ScrapeThread(queue, semaphore)
        t.start()
        print(f"Thread {i} started")
        threads.append(t)

    for j, url in enumerate(urls, start=1):
        print(f"Queueing URL {j}")
        semaphore.acquire()
        queue.put((url, j))
        print(f"Queue size is now: {queue.qsize()}")

    queue.join()
    print("All tasks have been processed")

    for i in range(len(threads)):
        print("Sending stop signal to threads")
        queue.put(None)
    for t in threads:
        print("Waiting for thread to finish")
        t.join()

if __name__ == "__main__":
    main(urls)

Exception in thread Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-12:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-13:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-14:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Thread-10:
Traceback (most recent call last):
  File "c:\Program Files\Pytho

Thread initialized
Thread running
Thread 0 started
Thread initialized
Thread running
Thread 1 started
Thread initialized
Thread running
Thread 2 started
Thread initialized
Thread running
Thread 3 started
Thread initialized
Thread running
Thread 4 started
Thread initialized
Thread running
Thread 5 started
Thread initialized
Thread running
Thread 6 started
Thread initialized
Thread running
Thread 7 started
Thread initialized
Thread running
Thread 8 started
Thread initialized
Thread running
Thread 9 started
Queueing URL 1
Queue size is now: 1
Queueing URL 2
Queue size is now: 2
Queueing URL 3
Queue size is now: 3
Queueing URL 4
Queue size is now: 4
Queueing URL 5
Queue size is now: 5
Queueing URL 6
Queue size is now: 6
Queueing URL 7
Queue size is now: 7
Queueing URL 8
Queue size is now: 8
Queueing URL 9
Queue size is now: 9
Queueing URL 10
Queue size is now: 10
Queueing URL 11
Error processing URL 1: name 'driver' is not defined
Releasing semaphore and marking task done
Error processing 